KeyError: 'is_open'

In [3]:
import os
import requests
import json
import time
from datetime import datetime, timedelta

API_KEY = 'PKDNTPXAIUV6X0CBLYUP'
SECRET_KEY = 'QXJ0g9aUwGAd50MwIeOVJ50V1h59NWO4ZYVOssgT'
BASE_URL = 'https://paper-api.alpaca.markets'

HEADERS = {
    'APCA-API-KEY-ID': API_KEY,
    'APCA-API-SECRET-KEY': SECRET_KEY
}

def get_account():
    url = f'{BASE_URL}/v2/account'
    r = requests.get(url, headers=HEADERS)
    return json.loads(r.content)

def get_asset_price(asset):
    url = f'{BASE_URL}/v2/assets/{asset}/last'
    r = requests.get(url, headers=HEADERS)
    return json.loads(r.content)

def create_order(symbol, qty, side, type, time_in_force):
    data = {
        'symbol': symbol,
        'qty': qty,
        'side': side,
        'type': type,
        'time_in_force': time_in_force
    }
    url = f'{BASE_URL}/v2/orders'
    r = requests.post(url, json=data, headers=HEADERS)
    return json.loads(r.content)

def main():
    invested = False
    while True:
        # Check if market is open
        clock_url = f'{BASE_URL}/v2/clock'
        r = requests.get(clock_url, headers=HEADERS)
        clock = json.loads(r.content)
        is_open = clock['is_open']

        if is_open and not invested:
            asset = 'AAPL'
            price = get_asset_price(asset)['last']['price']
            qty = int(111 // float(price))

            if qty > 0:
                create_order(asset, qty, 'buy', 'market', 'gtc')
                invested = True
                print(f'Bought {qty} shares of {asset} at {price}')

        if not is_open and invested:
            account = get_account()
            balance_change = float(account['equity']) - 111

            print(f'Passive income change: ${balance_change:.2f}')
            print(f'Total account value: ${account["equity"]}')
            break

        time.sleep(60)

if __name__ == '__main__':
    main()


ConnectionError: HTTPSConnectionPool(host='paper-api.alpaca.markets', port=443): Max retries exceeded with url: /v2/clock (Caused by NewConnectionError('<urllib3.connection.HTTPSConnection object at 0x000001BE550A3950>: Failed to establish a new connection: [Errno 11001] getaddrinfo failed'))